## GPT-2 GUI

In [ ]:
import torch
import torch.nn as nn
import math
from transformers import GPT2Tokenizer
import tkinter as tk
from tkinter import scrolledtext, messagebox

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

# 2. Multi-Head Self-Attention
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.scale = math.sqrt(self.d_k)

    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.size()
        Q = self.W_q(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, V)
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        output = self.W_o(context)
        return output

# 3. Feed-Forward Neural Network
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.linear2(self.relu(self.linear1(x)))

# 4. Transformer Decoder Layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadSelfAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        attn_output = self.self_attn(x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

# 5. GPT-2 Model
class GPT2(nn.Module):
    def __init__(self, vocab_size, d_model=768, num_layers=2, num_heads=12, d_ff=3072, max_len=256, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len)
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)
        ])
        self.output_layer = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def forward(self, x, mask=None):
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        for layer in self.decoder_layers:
            x = layer(x, mask)
        return self.output_layer(x)

    def generate(self, input_ids, max_length=50, method="greedy"):
        self.eval()
        with torch.no_grad():
            for _ in range(max_length):
                mask = torch.tril(torch.ones(input_ids.size(1), input_ids.size(1), device=device)).unsqueeze(0).unsqueeze(0)
                logits = self(input_ids, mask)[:, -1, :]
                if method == "greedy":
                    next_token = torch.argmax(logits, dim=-1, keepdim=True)
                else:
                    probs = torch.softmax(logits, dim=-1)
                    next_token = torch.multinomial(probs, num_samples=1)
                input_ids = torch.cat([input_ids, next_token], dim=1)
                if next_token.item() == tokenizer.eos_token_id:
                    break
        return input_ids

# 6. Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# 7. Load Model
def load_model(model, load_path):
    model.load_state_dict(torch.load(load_path, map_location=device))
    model.to(device)
    print(f"Model loaded from {load_path}")
    return model

# 8. Generate Text
def generate_text(model, prompt, max_length=50, method="greedy"):
    try:
        model.eval()
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
        generated_ids = model.generate(input_ids, max_length, method)
        return tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    except Exception as e:
        return f"Error generating text: {str(e)}"

# GUI Application
class GPT2GUI:
    def __init__(self, root, model):
        self.root = root
        self.model = model
        self.root.title("GPT-2 Text Generator")
        self.root.geometry("600x500")
        self.root.configure(bg="#E6F3FF")  # Light blue background

        # Center the window
        self.root.update_idletasks()
        width = self.root.winfo_width()
        height = self.root.winfo_height()
        x = (self.root.winfo_screenwidth() // 2) - (width // 2)
        y = (self.root.winfo_screenheight() // 2) - (height // 2)
        self.root.geometry(f"{width}x{height}+{x}+{y}")

        # Title Label
        title_label = tk.Label(
            root, text="GPT-2 Text Generator", font=("Helvetica", 16, "bold"),
            bg="#E6F3FF", fg="#003087"  # Dark blue text
        )
        title_label.pack(pady=10)

        # Prompt Label and Entry
        prompt_label = tk.Label(
            root, text="Enter Prompt:", font=("Helvetica", 12),
            bg="#E6F3FF", fg="#003087"
        )
        prompt_label.pack()
        self.prompt_entry = tk.Entry(
            root, width=50, font=("Helvetica", 10),
            bg="white", fg="black", bd=2, relief="groove"
        )
        self.prompt_entry.insert(0, "I have a cat")  # Default prompt
        self.prompt_entry.pack(pady=5)

        # Output Text Area with Scrollbar
        self.output_text = scrolledtext.ScrolledText(
            root, width=60, height=15, font=("Helvetica", 10),
            bg="white", fg="black", bd=2, relief="groove", wrap=tk.WORD
        )
        self.output_text.pack(pady=10)
        self.output_text.config(state="disabled")  # Read-only initially

        # Button Frame
        button_frame = tk.Frame(root, bg="#E6F3FF")
        button_frame.pack(pady=10)

        # Button Styles
        button_style = {
            "font": ("Helvetica", 10, "bold"), "relief": "raised", "bd": 3,
            "width": 12, "cursor": "hand2"
        }

        # New Prompt Button
        self.new_button = tk.Button(
            button_frame, text="New Prompt", command=self.new_prompt,
            bg="#4CAF50", fg="white", **button_style
        )
        self.new_button.grid(row=0, column=0, padx=5)
        self.new_button.bind("<Enter>", lambda e: self.new_button.config(bg="#45a049"))
        self.new_button.bind("<Leave>", lambda e: self.new_button.config(bg="#4CAF50"))

        # Generate Button
        self.generate_button = tk.Button(
            button_frame, text="Generate", command=self.generate,
            bg="#2196F3", fg="white", **button_style
        )
        self.generate_button.grid(row=0, column=1, padx=5)
        self.generate_button.bind("<Enter>", lambda e: self.generate_button.config(bg="#1e88e5"))
        self.generate_button.bind("<Leave>", lambda e: self.generate_button.config(bg="#2196F3"))

        # Close Button
        self.close_button = tk.Button(
            button_frame, text="Close", command=self.close,
            bg="#F44336", fg="white", **button_style
        )
        self.close_button.grid(row=0, column=2, padx=5)
        self.close_button.bind("<Enter>", lambda e: self.close_button.config(bg="#e53935"))
        self.close_button.bind("<Leave>", lambda e: self.close_button.config(bg="#F44336"))

        # Footer Label
        footer_label = tk.Label(
            root, text="GPT-2 Tiny Stories", font=("Helvetica", 10, "italic"),
            bg="#E6F3FF", fg="#003087"
        )
        footer_label.pack(side="bottom", pady=5)

    def new_prompt(self):
        self.prompt_entry.delete(0, tk.END)
        self.prompt_entry.insert(0, "")
        self.output_text.config(state="normal")
        self.output_text.delete("1.0", tk.END)
        self.output_text.config(state="disabled")

    def generate(self):
        prompt = self.prompt_entry.get().strip()
        if not prompt:
            messagebox.showwarning("Warning", "Please enter a prompt!")
            return

        try:
            generated_text = generate_text(self.model, prompt, max_length=50, method="greedy")
            self.output_text.config(state="normal")
            self.output_text.delete("1.0", tk.END)
            self.output_text.insert(tk.END, f"Prompt: {prompt}\n\nGenerated Text: {generated_text}")
            self.output_text.config(state="disabled")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to generate text: {str(e)}")

    def close(self):
        self.root.destroy()

# Main Execution
if __name__ == "__main__":
    # Initialize model
    vocab_size = tokenizer.vocab_size
    model = GPT2(vocab_size=vocab_size, d_model=768, num_layers=2, num_heads=12, d_ff=3072, max_len=256).to(device)

    # Load saved model
    load_path = r"D:\me\College\3rd year 2nd term\Pattern\final project\gpt2_model.pth" # Put the model path here.
    try:
        model = load_model(model, load_path)
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        exit(1)

    # Create GUI
    root = tk.Tk()
    app = GPT2GUI(root, model)
    root.mainloop()

Using device: cuda
Model loaded from D:\me\College\3rd year 2nd term\Pattern\final project\gpt2_model.pth
